# SUJET

Vous prévoyez des étiquettes de localisation des organites de protéines pour chaque échantillon. Au total, 28 étiquettes différentes sont présentes dans l'ensemble de données. L'ensemble de données est acquis de manière hautement normalisée en utilisant une modalité d'imagerie (microscopie confocale). Cependant, l'ensemble de données comprend 27 types de cellules de morphologie très différente, qui affectent les profils protéiques des différents organites. Tous les échantillons d'images sont représentés par quatre filtres (stockés sous forme de fichiers individuels), la protéine d'intérêt (vert) et trois repères cellulaires: noyau (bleu), microtubules (rouge), réticulum endoplasmique (jaune). Le filtre vert devrait donc être utilisé pour prédire l'étiquette, et les autres filtres sont utilisés comme références.

0.  Nucleoplasm  
1.  Nuclear membrane   
2.  Nucleoli   
3.  Nucleoli fibrillar center   
4.  Nuclear speckles   
5.  Nuclear bodies   
6.  Endoplasmic reticulum   
7.  Golgi apparatus   
8.  Peroxisomes   
9.  Endosomes   
10.  Lysosomes   
11.  Intermediate filaments   
12.  Actin filaments   
13.  Focal adhesion sites   
14.  Microtubules   
15.  Microtubule ends   
16.  Cytokinetic bridge   
17.  Mitotic spindle   
18.  Microtubule organizing center   
19.  Centrosome   
20.  Lipid droplets   
21.  Plasma membrane   
22.  Cell junctions   
23.  Mitochondria   
24.  Aggresome   
25.  Cytosol   
26.  Cytoplasmic bodies   
27.  Rods & rings  

# IMPORT

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from skimage.io import imread
import cv2
from skimage.transform import resize
from sklearn.model_selection import StratifiedKFold
from skimage.filters import try_all_threshold

In [ ]:
df=pd.read_csv('../input/train.csv', index_col=0)
df.head()

In [ ]:
images=[imread('../input/train/'+x+'_green.png', as_gray=True) for x in df.index]
print(images[0].shape)
df['Image']=images

In [ ]:
df.head()

# Green_IMAGE

In [ ]:
fig, ax = try_all_threshold(df.iloc[0, 1], figsize=(10, 8), verbose=False)
plt.show()

In [ ]:
shape=[1, 5]
fig=plt.figure(figsize=(20, 35))
for i in range(shape[0]*shape[1]):
    sub=plt.subplot(shape[0], shape[1], i+1)
    plt.imshow(df.iloc[i, 1])
#print('Example Images')
#plt.show()

In [ ]:
df['Hist']=df['Image'].apply(lambda x: np.histogram(x, 128)[0])
df.head()

In [ ]:
from sklearn.cluster import KMeans
clusters=10
kmeans=KMeans(clusters)
df['Cluster']=kmeans.fit_predict([np.array(x) for x in df['Hist'].values])
df.head()
shape=[1, 5]
fig=plt.figure(figsize=(20, 35))
for i in range(shape[0]*shape[1]):
    sub=plt.subplot(shape[0], shape[1], i+1)
    sub.set_title(df.iloc[i, 3])
    plt.imshow(df.iloc[i, 1])
print('Example Images')
plt.show()

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from PIL import Image


data_train_dir = r'../input/train'
answer_file_path = r'../input/train.csv'

channels = ['_yellow', '_red', '_green', '_blue']

index_class_dict = {
    0: "Nucleoplasm",
    1: "Nuclear membrane",
    2: "Nucleoli",
    3: "Nucleoli fibrillar center",
    4: "Nuclear speckles",
    5: "Nuclear bodies",
    6: "Endoplasmic reticulum",
    7: "Golgi apparatus",
    8: "Peroxisomes",
    9: "Endosomes",
    10: "Lysosomes",
    11: "Intermediate filaments",
    12: "Actin filaments",
    13: "Focal adhesion sites",
    14: "Microtubules",
    15: "Microtubule ends",
    16: "Cytokinetic bridge",
    17: "Mitotic spindle",
    18: "Microtubule organizing center",
    19: "Centrosome",
    20: "Lipid droplets",
    21: "Plasma membrane",
    22: "Cell junctions",
    23: "Mitochondria",
    24: "Aggresome",
    25: "Cytosol",
    26: "Cytoplasmic bodies",
    27: "Rods & rings"
}

train_df = pd.read_csv(answer_file_path)

In [ ]:
train_df[f'target_vec'] = train_df['Target'].map(lambda x: list(map(int, x.strip().split())))
for i in range(28):
    train_df[f'{index_class_dict[i]}'] = train_df['Target'].map(
             lambda x: 1 if str(i) in x.strip().split() else 0)
train_df.head()


In [ ]:
class_index = 1  # Nuclear membrane
current_part = train_df[train_df[index_class_dict[class_index]] == 1]

In [ ]:
#Functions for visualization
def make_rgb_image_from_four_channels(channels: list, image_width=512, image_height=512) -> np.ndarray:
    """
    It makes literally RGB image from source four channels, 
    where yellow image will be yellow color, red will be red and so on  
    """
    rgb_image = np.zeros(shape=(image_height, image_width, 3), dtype=np.float)
    yellow = np.array(Image.open(channels[0]))
    # yellow is red + green
    rgb_image[:, :, 0] += yellow/2   
    rgb_image[:, :, 1] += yellow/2
    # loop for R,G and B channels
    for index, channel in enumerate(channels[1:]):
        current_image = Image.open(channel)
        rgb_image[:, :, index] += current_image
    # Normalize image
    rgb_image = rgb_image / rgb_image.max() * 255
    return rgb_image.astype(np.uint8)
def visualize_part(start_class_index=0, nrows=4, ncols=3):
    """
    Visualize the part of classes, started from class with index start_class_index,
    make nrows classes, ncols examples for each one
    """
    fig, ax = plt.subplots(nrows = nrows, ncols=ncols, figsize=(15, 25))
    for class_index in range(nrows):
        current_index = class_index + start_class_index
        for sample in range(ncols):
            current_part = train_df[train_df[index_class_dict[current_index]] == 1] 
            # 0 index is id
            random_index = np.random.choice(current_part.values.shape[0], 1, replace=False)
            # random line from data with selected class
            current_line = current_part.values[random_index][0]
            image_names = [os.path.join(data_train_dir, current_line[0]) 
                           + x + '.png' for x in channels]
            rgb_image = make_rgb_image_from_four_channels(image_names)
            # text annotations, main title and subclasses (may be empty in case one label)
            main_class = index_class_dict[current_index]+'\n'
            # 2 index is vector with classes, split version of Target col
            other_classes = [index_class_dict[x] for x in current_line[2] 
                             if x != (current_index)]
            subtitle = ', '.join(other_classes)
            # show image
            ax[class_index, sample].set_title(main_class, fontsize=18)
            ax[class_index, sample].text(250, -10, subtitle, 
                                         fontsize=14, horizontalalignment='center')
            ax[class_index, sample].imshow(rgb_image)
            ax[class_index, sample].set_xticklabels([])
            ax[class_index, sample].set_yticklabels([])
            ax[class_index, sample].tick_params(left=False, bottom=False)

In [ ]:
#visualize 3 examples for each class (4*3)
visualize_part(0)

In [ ]:
#class study

import numpy as np
import pandas as pd 
import seaborn as sns
import os
from itertools import chain
from collections import Counter
sns.set_style("white")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

#from keras.preprocessing.image import load_img
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('../input/train.csv')
def count_target(target_val):
    return len(target_val.split(' '))

df['nclass'] = df.Target.map(count_target)

In [ ]:
#Function for plotting the images

def plot_img(img_id):
    filt_no = 0
    plt.figure(figsize=(30,60))
    plt.tight_layout()
    for filt in img_filt:
        img_path = train_img_path + img_id + filt
        img = np.array(load_img(img_path))
        plt.subplot(1,4,filt_no+1)
        plt.imshow(img)
        if i==0: plt.title(filt[1:-4]+' filter')
        plt.axis('off')
        filt_no += 1 

In [ ]:
#Label Count Distribribution

label_count = []
for i in range(df.nclass.min(),df.nclass.max()+1):
    label_count.append(np.sum(df.nclass==i))
    print('No. of images with',i,'label:',label_count[-1])

In [ ]:
x = np.arange(len(label_count))+1
plt.bar(x,label_count)
plt.title('Label Count Distribution in Train Set')

In [ ]:
#Mapping the labels

def mk_list(val):
    return [int(label) for label in val.split(' ')]
df['target_list'] = df['Target'].map(mk_list)
all_labels = list(chain.from_iterable(df['target_list'].values))
label_count = Counter(all_labels)
arr = np.zeros((28,))
for key,value in label_count.items():
    arr[key] = value

In [ ]:
map_class_labels = {
0:  'Nucleoplasm',
1:  'Nuclear membrane',
2:  'Nucleoli',
3:  'Nucleoli fibrillar center',
4:  'Nuclear speckles',
5:  'Nuclear bodies',
6: 'Endoplasmic reticulum',
7:  'Golgi apparatus',
8:  'Peroxisomes',
9:  'Endosomes',
10:  'Lysosomes',
11:  'Intermediate filaments',
12:  'Actin filaments',
13:  'Focal adhesion sites',
14: 'Microtubules',
15: 'Microtubule ends',
16: 'Cytokinetic bridge',
17: 'Mitotic spindle',
18: 'Microtubule organizing center',
19: 'Centrosome',
20: 'Lipid droplets',
21: 'Plasma membrane',
22: 'Cell junctions',
23: 'Mitochondria',
24: 'Aggresome',
25: 'Cytosol',
26: 'Cytoplasmic bodies',
27: 'Rods and rings'}

In [ ]:
#graph class 

plt.figure(figsize=(30,5))
ax = sns.barplot(x=np.arange(28),y=arr)
ax.set_xticklabels(list(map_class_labels.values()), fontsize=15, rotation=40, ha="right")
ax.set(xlabel='Classes', ylabel='Class Counts')
plt.show()

In [ ]:
#Plotting the image with the histogram
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.gray()
import skimage.filters
import scipy.ndimage
import sklearn.feature_extraction
import sklearn.cluster
import skimage.feature
import skimage.transform
import PIL
import sys



In [ ]:
def plot_file(filename):
    image = plt.imread(filename)
    hist = np.histogram(image - image.mean(),
                        bins=np.arange(image.min(),
                                       image.max(),
                                       1/256))

    fig, axes = plt.subplots(1, 2, figsize=(8, 3))
    
    axes[0].imshow(image, interpolation='nearest')
    axes[0].axis('off')
    axes[0].set_title(filename[-20:])
    
    axes[1].plot(hist[1][:-1], hist[0], lw=2)
    axes[1].set_title('histogram of gray values')
    
    plt.show()

In [ ]:
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_red.png')
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_green.png')
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_blue.png')
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_yellow.png')

In [ ]:
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_red.png')
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_green.png')
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_blue.png')
plot_file('../input/train/002daad6-bbc9-11e8-b2bc-ac1f6b6435d0_yellow.png')


In [ ]:
#Other

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import itertools as itt
import networkx as nx
from sklearn.preprocessing import MinMaxScaler


In [ ]:
train_labels = pd.read_csv("../input/train.csv")

In [ ]:
label_names = {
    0:  "Nucleoplasm",  
    1:  "Nuclear membrane",   
    2:  "Nucleoli",   
    3:  "Nucleoli fibrillar center",   
    4:  "Nuclear speckles",
    5:  "Nuclear bodies",   
    6:  "Endoplasmic reticulum",   
    7:  "Golgi apparatus",   
    8:  "Peroxisomes",   
    9:  "Endosomes",   
    10:  "Lysosomes",   
    11:  "Intermediate filaments",   
    12:  "Actin filaments",   
    13:  "Focal adhesion sites",   
    14:  "Microtubules",   
    15:  "Microtubule ends",   
    16:  "Cytokinetic bridge",   
    17:  "Mitotic spindle",   
    18:  "Microtubule organizing center",   
    19:  "Centrosome",   
    20:  "Lipid droplets",   
    21:  "Plasma membrane",   
    22:  "Cell junctions",   
    23:  "Mitochondria",   
    24:  "Aggresome",   
    25:  "Cytosol",   
    26:  "Cytoplasmic bodies",   
    27:  "Rods & rings"
}

reverse_train_labels = dict((v,k) for k,v in label_names.items())

def fill_targets(row):
    row.Target = np.array(row.Target.split(" ")).astype(np.int)
    for num in row.Target:
        name = label_names[int(num)]
        row.loc[name] = 1
    return row

In [ ]:
for key in label_names.keys():
    train_labels[label_names[key]] = 0

In [ ]:
train_labels = train_labels.apply(fill_targets, axis=1)

In [ ]:
target_counts = train_labels.drop(["Id", "Target"],axis=1).sum(axis=0).sort_values(ascending=False)
plt.figure(figsize=(15,15))
sns.barplot(y=target_counts.index.values, x=target_counts.values, order=target_counts.index)

In [ ]:
target_counts.tail()

In [ ]:
train_labels["number_of_targets"] = train_labels.drop(["Id", "Target"],axis=1).sum(axis=1)

In [ ]:
count_perc = train_labels.groupby("number_of_targets").count()['Id']
count_perc

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(count_perc,
        labels=["%d targets" % x for x in count_perc.index],
        autopct='%1.1f%%')
plt.ylabel('');

In [ ]:
#Other
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# number of validation images to use
VALID_IMG_COUNT = 1000
# maximum number of training images
MAX_TRAIN_IMAGES = 15000 
BASE_MODEL='RESNET52' # ['VGG16', 'RESNET52', 'InceptionV3', 'Xception', 'DenseNet169', 'DenseNet121']
IMG_SIZE = (299, 299) # [(224, 224), (384, 384), (512, 512), (640, 640)]
BATCH_SIZE = 32 # [1, 8, 16, 24]
DROPOUT = 0.5
DENSE_COUNT = 128
LEARN_RATE = 1e-4
EPOCHS = 10
RGB_FLIP = 1 # should rgb be flipped when rendering images

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util import montage
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
base_dir = '../input'
train_image_dir = os.path.join(base_dir, 'train')
test_image_dir = os.path.join(base_dir, 'test')
import gc; gc.enable() # memory is tight
import numpy as np

In [ ]:
image_df = pd.read_csv(os.path.join('../input/',
                                 'train.csv'))
print(image_df.shape[0], 'masks found')
print(image_df['Id'].value_counts().shape[0])
# just use green for now
image_df['Green_path'] = image_df['Id'].map(lambda x: os.path.join(train_image_dir, '{}_green.png'.format(x)))
image_df['Target_list'] = image_df['Target'].map(lambda x: [int(a) for a in x.split(' ')])


In [ ]:
from itertools import chain
from collections import Counter
all_labels = list(chain.from_iterable(image_df['Target_list'].values))
c_val = Counter(all_labels)
n_keys = c_val.keys()
max_idx = max(n_keys)
fig, ax1 = plt.subplots(1,1, figsize = (10, 5))
ax1.bar(n_keys, [c_val[k] for k in n_keys])
for k,v in c_val.items():
    print(k, 'count:', v)

In [ ]:
# create a categorical vector
image_df['Target_vec'] = image_df['Target_list'].map(lambda ck: [i in ck for i in range(max_idx+1)])
image_df.sample(3)

In [ ]:
image_df.shape

In [ ]:
#WHYNOT

In [ ]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(image_df, 
                 test_size = 0.3, 
                  # hack to make stratification work                  
                 stratify = image_df['Target'].map(lambda x: x[:3] if '27' not in x else '0'))
print(train_df.shape[0], 'Training masks')
print(valid_df.shape[0], 'Validation masks')

In [ ]:
train_df = train_df.sample(min(MAX_TRAIN_IMAGES, train_df.shape[0])) # limit size of training set (otherwise it takes too long)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
train_sum_vec = np.sum(np.stack(train_df['Target_vec'].values, 0), 0)
valid_sum_vec = np.sum(np.stack(valid_df['Target_vec'].values, 0), 0)
ax1.bar(n_keys, [train_sum_vec[k] for k in n_keys])
ax1.set_title('Training Distribution')
ax2.bar(n_keys, [valid_sum_vec[k] for k in n_keys])
ax2.set_title('Validation Distribution')

In [ ]:
test=imread('../input/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0_green.png', as_gray=True)
test

***START***

In [ ]:
#image visualisation 
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from PIL import Image

In [ ]:
#load data
train_df = pd.read_csv('../input/train.csv')
train_df.head()

In [ ]:
#target = train_df["Target"]
labels = train_df["Target"]
ids = train_df["Id"]

In [ ]:
#creation of classes
channels = ['_yellow', '_red', '_green', '_blue']

index_class_dict = {
   # "Nucleoplasm": 0,
    0: 0,
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    8: 8,
    9: 9,
    10: 10,
    11: 11,
    12: 12,
    13: 13,
    14: 14,
    15: 15,
    16: 16,
    17: 17,
    18: 18,
    19: 19,
    20: 20,
    21: 21,
    22: 22,
    23: 23,
    24: 24,
    25: 25,
    26: 26,
    27: 27
}

dict = {v: k for k, v in index_class_dict.items()}

def to_cat_array(label_list):
        result = np.zeros(len(index_class_dict), dtype = np.float32)
        for label in label_list:
            result[dict[label]] = 1.0
        return result
    
#index_class_dict

In [ ]:
train_df[f'Target_Vec'] = train_df['Target'].map(lambda x: list(map(int, x.strip().split())))
train_df.head()

In [ ]:
from skimage.io import imread
images=[imread('../input/train/'+x+'_green.png', as_gray=True) for x in train_df.Id]
print(images[0].shape)
train_df['Image']=images

In [ ]:
train_df.head()

In [ ]:
#ici

In [ ]:
#test
fake_label_lists = [[0, 2], [4, 10], [3, 8],[5, 12, 22]]

In [ ]:
categories = list (map(to_cat_array, fake_label_lists))
categories

In [ ]:
train_df['Label'] = list(map(to_cat_array, train_df.Target_Vec))

In [ ]:
train_df.head()

WORK ON ONLY 500 IMAGES

In [ ]:
images = train_df.Image.iloc[0:500]

In [ ]:
images.head()

In [ ]:
labels = train_df.Label.iloc[0:500]

In [ ]:
labels.head()

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
from sklearn.model_selection import train_test_split
train_images, test_images,train_labels, test_labels = train_test_split(images, labels, train_size=0.8, random_state=0)

In [ ]:
img=train_images.iloc[4]
plt.imshow(img,cmap='gray')

In [ ]:
plt.imshow(img,cmap='binary')

In [ ]:
#from sklearn.datasets import load_iris
#from sklearn.model_selection import cross_val_score
#from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier(random_state=0)
#iris = load_iris()
#clf.fit([np.array(image, dtype=np.int8).reshape(512*512) for image in train_images], [np.array(label, dtype=np.int64) for label in train_labels])
#clf.score([np.array(image, dtype=np.int8).reshape(512*512) for image in train_images], [np.array(label, dtype=np.int64) for label in train_labels])

In [ ]:
[np.array(image, dtype=np.int8).reshape(512*512) for image in train_images]

In [ ]:
[np.array(label, dtype=np.int64) for label in train_labels]

In [ ]:
#echec
#cross_val_score(clf, [np.array(image, dtype=np.int8).reshape(512*512) for image in train_images], [np.array(label, dtype=np.int64) for label in train_labels], cv=10)